## Imports

In [2]:
from transformers import CLIPProcessor, CLIPModel
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import pandas as pd
import os


/screenrpa/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fine-tuning

### Lectura de datos

In [3]:
def read_ui_log_as_dataframe(log_path):
  return pd.read_csv(log_path, sep=";")#, index_col=0)

In [4]:
df = read_ui_log_as_dataframe('resources/sc_0_size50_Balanced/log_m.csv')

### Clase CustomClipDataset
Definimos nuestro conjunto de datos personalizado en el que cargamos imágenes y sus etiquetas de texto asociadas desde un Dataframe.
Invocamos el procesor de CLIP para preparar las imágenes y textos para el modelo.
Transformarmos imágenes y textos a tensores.

In [ ]:
class CustomCLIPDataset(Dataset):
    def __init__(self, dataframe, processor):
        self.dataframe = dataframe
        self.processor = processor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join('resources', 'sc_0_size50_Balanced', row['Screenshot'])
        text_label = row['header']  
        image = Image.open(image_path).convert("RGB")
        inputs = self.processor(text=[text_label], images=image, return_tensors="pt", padding=True)

        return inputs['pixel_values'].squeeze(), inputs['input_ids'].squeeze()

Iniciación del modelo clip y del procesador con los pesos preentenados del propio modelo de OpenAI. 
Estos componentes extraen las caracterísiticas visuales y lingüisticas de los datos. Con el dataset, creamos la instancia del conjunto de datos que con tiene las rutas a las imágenes y textos asociados, junto con el procesador de CLIP.

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
dataset = CustomCLIPDataset(df, processor)

Función de preprocesamiento de los datos. Es necesario que los textos tengan la misma longitud para su procesamiento. Se aplica padding para ello. Se facilita el entrenamiento por lotes.

In [ ]:
def collate_fn(batch):
    images, texts = zip(*batch)
    images_stacked = torch.stack(images)
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=processor.tokenizer.pad_token_id)
    
    return images_stacked, texts_padded

Instanciación de DataLoader que permite automatizar la carga y preparación de datos para el entrenamiento a partir de la función que definimos anteriormente.
Aparte, también definimos la función de perdida (CrossEntropyLoss) y un optimizador del entrenamiento con una taza de aprendizaje baja.

In [ ]:
data_loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Entrenamiento del modelo CLIP por épocas. Ajustar el número correspondiente en el bucle.
Este tipo de entrenamiento a través de épocas buscas reducir la pérdida ajustando el modelo para correlacionar las imágenes con los textos.

In [6]:
model.train()
for epoch in range(10): 
    for images, texts in data_loader:
        images, texts = images.to(model.device), texts.to(model.device)
        outputs = model(pixel_values=images, input_ids=texts)
        loss = loss_fn(outputs.logits_per_image, outputs.logits_per_text.argmax(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch}: Loss {loss.item()}")
model.save_pretrained("resources/my_finetuned_clip")

/screenrpa/venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Epoch 0: Loss 0.7255793809890747
Epoch 1: Loss 0.97235107421875
Epoch 2: Loss 0.5263630151748657
Epoch 3: Loss 0.19276100397109985
Epoch 4: Loss 0.1865340769290924
Epoch 5: Loss 0.6931476593017578
Epoch 6: Loss 0.6931471824645996
Epoch 7: Loss 0.45485052466392517
Epoch 8: Loss 0.6931471824645996
Epoch 9: Loss 0.6931471824645996
